In [18]:
from google.colab import drive
drive.mount('/content/drive')


#!/usr/bin/python
#read data set to be used

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#load dataset

dataset=pd.read_csv('drive/MyDrive/Colab Notebooks/Data/datafinal.csv')
dataset['timeIndex']= (dataset['newTime']+1)/288

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (133) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Generate price clasess

In [76]:
dataset['classes']= pd.cut(dataset['rrp_x'],bins=[-1000,25,70,15001],include_lowest=True,right=False, labels=['vl','normal','high'])
#generate onehot encoder

data_encoded1=pd.get_dummies(dataset['classes'],prefix='PriceQLD')
print(data_encoded1.columns)
datasetfinal=pd.concat([dataset,data_encoded1],axis=1)

Index(['PriceQLD_vl', 'PriceQLD_normal', 'PriceQLD_high'], dtype='object')


SELECT VARIABLES

In [84]:
#generate x and y
#y will be the price

y= datasetfinal.loc[:,datasetfinal.columns.isin(['PriceQLD_vl', 'PriceQLD_normal', 'PriceQLD_high'])]

x= datasetfinal.loc[:,datasetfinal.columns.isin(['PriceQLD_vl', 'PriceQLD_normal', 'PriceQLD_high','timeIndex','month','day_of_week','rrp_y','RMNSW','totaldemand_y','initialsupply_x','dispatchablegeneration_x','netinterchange_x'])]

FUNCTION FOR DIVISION IN TEST AND TRAIN SAMPLES, THE

In [78]:

def train_test_split(x,y, n_test):
	return x[:-n_test], x[-n_test:],y[:-n_test],y[-n_test:]
1
2


2

DIVISION IN TEST AND TRAIN SAMPLES

In [85]:
n_test=80000
X_train, X_test, Y_train, Y_test=train_test_split(x,y, n_test)

n= time horizon
neurons= number of neurons in hiddent layer

In [86]:
#n=[6]
n=[6]
neurons=[150]
#n=[2,3]
#neurons=[10,20]

ROBUST SCALER

In [87]:
from sklearn.preprocessing import RobustScaler
# The values of the quantile range need to be tuned for better results
scale=RobustScaler(with_centering=False,quantile_range=(0.5,75))
scaleX = RobustScaler().fit(X_train)
xnorm= scaleX.transform(X_train)
#scaleY=RobustScaler().fit(Y_train)
import tensorflow as tf
y_train = Y_train
ytestnorm = Y_test
xtestnorm= scaleX.transform(X_test)
ynorm=y_train


(100868, 9)
[[[0 0 1]
  [0 0 1]]

 [[0 0 1]
  [0 1 0]]

 [[0 1 0]
  [0 0 1]]

 ...

 [[0 1 0]
  [0 1 0]]

 [[0 1 0]
  [0 1 0]]

 [[0 1 0]
  [0 1 0]]]


In [48]:
print(ynorm)

        PriceQLD_vl  PriceQLD_normal  PriceQLD_high
0                 0                0              1
1                 0                0              1
2                 0                0              1
3                 0                0              1
4                 0                0              1
...             ...              ...            ...
100863            0                1              0
100864            0                1              0
100865            0                1              0
100866            0                1              0
100867            0                1              0

[100868 rows x 3 columns]


SPLIT TIME SEQUENCE

In [28]:
#do a function to split the time sequence to predict m Dispatch intervals using n samples in the past

#do a function to split the time sequence to predict m Dispatch intervals using n samples in the past at each step

def split_sequence(xin,yout, n, m):
    X, y = list(), list()
    for i in range(len(yout)):
# find the end of this pattern
        end_ix = i + n
        out_end_ix = end_ix + m 
# check if we are beyond the sequence
        if out_end_ix > len(yout):
            break
# gather input and output parts of the pattern
        seq_x = xin[i:end_ix]
        seq_y=yout[end_ix:out_end_ix]

        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

ITERATE THROUGH DIFFERENT VARIABLES AND TRAIN AND TEST NETWORK

In [ ]:
from numpy import array

recallV=list()
misclasV=list()
for i in n:
  for j in neurons:
    print(xnorm.shape)
    Xtrain2, ytrain2 = split_sequence(xnorm,ynorm,i,1)
    print(ytrain2)
    Xtestf, ytestf = split_sequence(xtestnorm,ytestnorm,i,1)
    print(ytrain2.shape)
    from keras.layers import Dense
    from keras.models import Sequential
    from keras.layers import Dropout
    import tensorflow as tf
    import keras as ks
    n_input = Xtrain2.shape[1] * Xtrain2.shape[2]
    ny=ytrain2.shape[1] * ytrain2.shape[2]
    X = Xtrain2.reshape(Xtrain2.shape[0],n_input)
    ytrain3=ytrain2.reshape(ytrain2.shape[0], ny)
    optimizer = ks.optimizers.Adagrad(lr=0.01)
    from keras.callbacks import ReduceLROnPlateau
    from keras.callbacks import EarlyStopping
    callbackEarlyStop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)


    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.95,
                              patience=5, min_lr=0.0000001,verbose=1)
    # define model
    model = Sequential()
    model.add(Dense(j, activation='relu', input_dim=n_input))
    model.add(Dropout(0.12))
    model.add(Dense(j, activation='relu'))
    model.add(Dropout(0.12))
    #model.add(Dense(j, activation='relu'))
    #model.add(Dropout(0.12))
    model.add(Dense(3,activation='softmax'))
    model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])

    model.fit(X, ytrain3,epochs=1054,batch_size=512,shuffle=False,validation_split=0.2,callbacks=[reduce_lr,callbackEarlyStop], verbose=2)
    


   
    print(X_test.shape)
    print(Y_test.shape)
    
    n_input2 = Xtestf.shape[1] * Xtestf.shape[2]

    Xtestf= Xtestf.reshape(Xtestf.shape[0], n_input2)
    ytestf=ytestf.reshape(ytestf.shape[0], ny)
    yhat = model.predict(Xtestf, verbose=0)
    ypred=yhat[:,0]
    ytestf2=ytestf[:,0]
    from sklearn.metrics import confusion_matrix
    print(confusion_matrix(ytestf.argmax(axis=1),yhat.argmax(axis=1) ))
    from sklearn.metrics import recall_score
    print('recall', recall_score( ytestf.argmax(axis=1), yhat.argmax(axis=1),average=None))
    recallV.append(recall_score(  ytestf.argmax(axis=1), yhat.argmax(axis=1),average=None))
    from sklearn.metrics import zero_one_loss
  #misclasV=[misclasV zero_one_loss(Y_testNP.argmax(axis=1), ypredNP.argmax(axis=1))]
    print('misclass',zero_one_loss(ytestf.argmax(axis=1), yhat.argmax(axis=1)))
    misclasV.append(zero_one_loss( ytestf.argmax(axis=1), yhat.argmax(axis=1)))



  
print('recall',recallV)
print('misclassification',misclasV)

(100868, 12)
[[[0 0 1]]

 [[0 1 0]]

 [[0 1 0]]

 ...

 [[0 1 0]]

 [[0 1 0]]

 [[0 1 0]]]
(100862, 1, 3)
Epoch 1/1054
158/158 - 2s - loss: 0.6831 - accuracy: 0.8111 - val_loss: 0.3674 - val_accuracy: 0.8809
Epoch 2/1054
158/158 - 1s - loss: 0.4123 - accuracy: 0.8833 - val_loss: 0.2826 - val_accuracy: 0.9075
Epoch 3/1054
158/158 - 1s - loss: 0.3610 - accuracy: 0.8920 - val_loss: 0.2553 - val_accuracy: 0.9150
Epoch 4/1054
158/158 - 1s - loss: 0.3755 - accuracy: 0.8977 - val_loss: 0.2446 - val_accuracy: 0.9161
Epoch 5/1054
158/158 - 1s - loss: 0.3290 - accuracy: 0.8994 - val_loss: 0.2424 - val_accuracy: 0.9168
Epoch 6/1054
158/158 - 1s - loss: 0.3196 - accuracy: 0.9019 - val_loss: 0.2380 - val_accuracy: 0.9174
Epoch 7/1054
158/158 - 1s - loss: 0.3118 - accuracy: 0.9025 - val_loss: 0.2367 - val_accuracy: 0.9184
Epoch 8/1054
158/158 - 1s - loss: 0.3057 - accuracy: 0.9037 - val_loss: 0.2328 - val_accuracy: 0.9189
Epoch 9/1054
158/158 - 1s - loss: 0.3021 - accuracy: 0.9035 - val_loss: 0.2320

In [109]:
 print(yhat.argmax(axis=1) )
 confusion_matrix(ytestf.argmax(axis=1),yhat.argmax(axis=1) )
print('recall', recall_score( ytestf.argmax(axis=1), yhat.argmax(axis=1),average=None))
print('misclass',zero_one_loss(ytestf.argmax(axis=1), yhat.argmax(axis=1)))

[1 1 1 ... 1 1 1]
recall [0.54682384 0.96043016 0.78464339]
misclass 0.07588069105182893
